# Gentle introduction to Ray datasets APIs

© 2019-2022, Anyscale. All Rights Reserved

### Learning objectives

In this introductory tutorial you will learn to:
 * create, transform, read, and save Ray datasets
 * use shards for parallel processing of large datasets
 * understand datapipelines and their merits
 * use `DatasetPipeline` for last-mile ML ingestion for distributed training
 * why use datasets and for what

### Overview

This is a brief and gentle introduction to Ray's native library `ray dataset`. As a native Ray library, built atop Ray, it allows you to exchange data among Ray tasks, actors, libraries, and applications. 

Ray Datasets, using distributed Apache Arrow, are designed to load and preprocess data for distributed ML training pipelines. Compared to other loading solutions, Datasets are more flexible (e.g., you can express higher-quality per-epoch global shuffles) and provides higher overall performance.

Additionally, Ray datasets provides standard transformations like `map`, `filter`, and `partition`. Ray datasets is *not* a replacement for a full-fledged data processing library for EDA, ETL or a subsitute for Apache Spark or Dask or Pandas DataFrames. Its primary objective is the last-mile rudimentary distributed data preprocessing and data ingestion for ML training.

Supporting myriad [file formats and data sources](https://docs.ray.io/en/latest/data/dataset.html#datasource-compatibility), you can read from and write to local FS and cloud storage. 

<img src="images/dataset.png" width="70%" height="35%">


### Key concepts

To work with Ray Datasets, you need to understand how Datasets and Dataset Pipelines work. You might also be interested to learn about the execution model of Ray Datasets operations.

#### Ray Datasets

A Ray dataset implements a distributed [Apache Arrow](https://arrow.apache.org/). A Dataset consists of a list of Ray object references to blocks. Each block holds a set of items in either an [Arrow table](https://arrow.apache.org/docs/python/data.html#tables) or a Python list (for Arrow incompatible objects).

<img src="images/dataset-arch.png" width="70%" height="35%">

#### Dataset Pipelines
Datasets execute their transformations synchronously in blocking calls. However, it can be useful to overlap dataset computations with output. This can be done with a `DatasetPipeline`.

A `DatasetPipeline` is an unified iterator over a (potentially infinite) sequence of Ray Datasets, each of which represents a window over the original data. Conceptually, it is similar to a `Spark DStream`, but manages execution over a bounded amount of source data instead of an unbounded stream. Ray computes each dataset window on-demand and stitches their output together into a single logical data iterator. `DatasetPipeline` implements most of the same transformation and output methods as Datasets (e.g., `map`, `filter`, `split`, `iter_rows`, `to_torch`, etc.).

### Datasets Execution Model
This section overviews the execution model of Datasets, which may be useful for understanding and tuning performance.

#### Reading Data
Datasets uses Ray tasks to read data from remote storage. When reading from a file-based datasource (e.g., S3, GCS), it creates a number of read tasks equal to the specified read parallelism (200 by default). One or more files will be assigned to each read task. Each read task reads its assigned files and produces one or more output blocks (Ray objects):

<img src="https://docs.ray.io/en/master/_images/dataset-read.svg" height="25%" width="50%">

In the common case, each read task produces a single output block. Read tasks may split the output into multiple blocks if the data exceeds the target max block size (2GiB by default). This automatic block splitting avoids out-of-memory errors when reading very large single files (e.g., a 100-gigabyte CSV file). All of the built-in datasources except for JSON currently support automatic block splitting.

#### Deferred Read Task Execution

When a Dataset is created using `ray.data.read_*`, only the first read task will be executed initially. This avoids blocking Dataset creation on the reading of all data files, enabling inspection functions like `ds.schema()` without incurring high read costs. `<ray.data.Dataset.schema>`() and `ds.show()` to be used right away. Executing further transformations on the Dataset will trigger execution of all read tasks.

#### Dataset Transforms

Datasets use either Ray tasks or Ray actors to transform datasets (i.e., for `ds.map_batches()`, `ds.map()`, or `ds.flat_map()`). By default, tasks are used `(compute="tasks")`. Actors can be specified with `compute="actors"`, in which case an autoscaling pool of Ray actors will be used to apply transformations. Using actors allows for expensive state initialization (e.g., for GPU-based tasks) to be re-used. Whichever compute strategy is used, each map task generally takes in one block and produces one or more output blocks. The output block splitting rule is the same as for file reads (blocks are split after hitting the target max block size of 2GiB):

<img src="https://docs.ray.io/en/master/_images/dataset-map.svg" height="25%" width="50%">

#### Shuffling Data

Certain operations like `ds.sort()` and `ds.groupby()` require data blocks to be partitioned by value. Datasets executes this in three phases. First, a wave of sampling tasks determines suitable partition boundaries based on a random sample of data. Second, map tasks divide each input block into a number of output blocks equal to the number of reduce tasks. Third, reduce tasks take assigned output blocks from each map task and combines them into one block. Overall, this strategy generates O(n^2) intermediate objects where n is the number of input blocks.

You can also change the partitioning of a Dataset using `ds.random_shuffle()` or `ds.repartition()`. The former should be used if you want to randomize the order of elements in the dataset. The second should be used if you only want to equalize the size of the Dataset blocks (e.g., after a read or transformation that may skew the distribution of block sizes). Note that repartition has two modes, `shuffle=False`, which performs the minimal data movement needed to equalize block sizes, and `shuffle=True`, which performs a full (non-random) distributed shuffle:

<img src="https://docs.ray.io/en/master/_images/dataset-shuffle.svg" height="25%" width="50%">

#### Fault tolerance

Datasets relies on task-based [fault tolerance](https://docs.ray.io/en/latest/ray-core/tasks/fault-tolerance.html) in Ray core. Specifically, a `Dataset` will be automatically recovered by Ray in case of failures. This works through **lineage reconstruction**: a Dataset is a collection of Ray objects stored in shared memory, and if any of these objects are lost, then Ray will recreate them by re-executing the task(s) that created them.

There are a few cases that are not currently supported: 1. If the original creator of the Dataset dies. This is because the creator stores the metadata for the objects that comprise the Dataset. 2. `For a DatasetPipeline.split()`, we do not support recovery for a consumer failure. When there are multiple consumers, they must all read the split pipeline in lockstep. To recover from this case, the pipeline and all consumers must be restarted together. 3. The `compute=actors` option for transformations.

#### Execution and Memory Management

See [Execution and Memory Management](https://docs.ray.io/en/master/data/memory-management.html#data-advanced) for more details about how Datasets manages memory and optimizations such as lazy vs eager execution.

In [1]:
import logging, random
import ray

In [2]:
if ray.is_initialized:
    ray.shutdown()
ctx = ray.init(logging_level=logging.ERROR)
print(ctx)

RayContext(dashboard_url='127.0.0.1:8266', python_version='3.8.13', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-06-15_14-07-12_383754_96365/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-06-15_14-07-12_383754_96365/sockets/raylet', 'webui_url': '127.0.0.1:8266', 'session_dir': '/tmp/ray/session_2022-06-15_14-07-12_383754_96365', 'metrics_export_port': 64743, 'gcs_address': '127.0.0.1:62332', 'address': '127.0.0.1:62332', 'node_id': 'fb23fee3340b65e63a0b0a138e8e21273b25955acc589cf95d103bd3'})


In [3]:
print(f"Dashboard url: http://{ctx.address_info['webui_url']}")

Dashboard url: http://127.0.0.1:8266


Let's create a generic dataset of 100K integers and look at the schema and underlying datatype. The difference between `show` and `take` is that the former takes one item at time and prints it, while the latter iterates over row items from the dataset, appends to a list and returns it. Underneath, `ds.show()` calls `ds.take()`.

In [4]:
ds = ray.data.range(100_000)
ds.count()

100000

In [5]:
ds.schema()

int

In [6]:
ds.show(5)

0
1
2
3
4


In [7]:
ds.take(5)

[0, 1, 2, 3, 4]

Let's create a synthetic dataset of Arrow records (750K) with several columns and data associated with it. 

In [8]:
STATES = ["CA", "AZ", "OR", "WA", "TX", "UT"]
M_STATUS = ["married", "single", "domestic", "divorced", "undeclared"]
GENDER = ["F", "M", "U"]
HOME_OWNER = ["condo", "house", "rental"]

items = [{"id": i,
          "ssn": None,
          "name": None,
          "amount": i * 1.5, 
          "interest": random.randint(1,5) * .1,
          "state": random.choice(STATES),
          "marital_status": random.choice(M_STATUS),
          "property": random.choice(HOME_OWNER),
          "dependents": random.randint(1, 5),
          "defaulted": random.randint(0,1),
          "gender":random.choice(GENDER) } for i in range(1,750_001)]
items[:2]

[{'id': 1,
  'ssn': None,
  'name': None,
  'amount': 1.5,
  'interest': 0.1,
  'state': 'CA',
  'marital_status': 'married',
  'property': 'rental',
  'dependents': 3,
  'defaulted': 1,
  'gender': 'F'},
 {'id': 2,
  'ssn': None,
  'name': None,
  'amount': 3.0,
  'interest': 0.4,
  'state': 'CA',
  'marital_status': 'divorced',
  'property': 'house',
  'dependents': 3,
  'defaulted': 0,
  'gender': 'M'}]

### Creating a dataset from list of dictionary items

In [9]:
arrow_ds = ray.data.from_items(items)
arrow_ds

Dataset(num_blocks=200, num_rows=750000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string})

In [10]:
arrow_ds.count()

750000

In [11]:
arrow_ds.take(1)

[ArrowRow({'id': 1,
           'ssn': None,
           'name': None,
           'amount': 1.5,
           'interest': 0.1,
           'state': 'CA',
           'marital_status': 'married',
           'property': 'rental',
           'dependents': 3,
           'defaulted': 1,
           'gender': 'F'})]

In [12]:
arrow_ds.schema

<bound method Dataset.schema of Dataset(num_blocks=200, num_rows=750000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string})>

### Saving datasets and reading as a parquet file
Ray datasets support myriad data formats and public storage. Let's save this dataset as a parquet file and create `N` partitions

In [13]:
arrow_ds.repartition(5).write_parquet("data/interest.parquet")

Write Progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 11.72it/s]


In [14]:
!ls -l data/interest.parquet

total 20856
-rw-r--r--  1 jules  staff  2139572 Jun 15 14:08 2d38b9d0f4ec45fbb148fc955112c817_000000.parquet
-rw-r--r--  1 jules  staff  2121380 Jun 15 14:08 2d38b9d0f4ec45fbb148fc955112c817_000001.parquet
-rw-r--r--  1 jules  staff  2119605 Jun 15 14:08 2d38b9d0f4ec45fbb148fc955112c817_000002.parquet
-rw-r--r--  1 jules  staff  2119220 Jun 15 14:08 2d38b9d0f4ec45fbb148fc955112c817_000003.parquet
-rw-r--r--  1 jules  staff  2169554 Jun 15 14:08 2d38b9d0f4ec45fbb148fc955112c817_000004.parquet


In [15]:
arrow_ds = ray.data.read_parquet("data/interest.parquet")

In [16]:
arrow_ds.take(1)

[ArrowRow({'id': 1,
           'ssn': None,
           'name': None,
           'amount': 1.5,
           'interest': 0.1,
           'state': 'CA',
           'marital_status': 'married',
           'property': 'rental',
           'dependents': 3,
           'defaulted': 1,
           'gender': 'F'})]

### Transforming data with simple methods

Ray datasets support transformation in parallel using `map`. It uses Ray tasks to execute eagerly or synchronously. Among others [transformations](https://docs.ray.io/en/latest/data/package-ref.html#dataset-api), it supports`filter`, `flat_map`, `groupBy`etc.

Let's try a using `.map()`, `.filter()` and `.groupBy` on our dataset. `map()` and `filter()` are
row-based operations. This can be expensive. However you can use `map_batches(...)` with batch_size=4096 as default. This will create a task per block and each batch will be vectorized and executed in parallel. Ray tasks are created per block for a map operation. 

Let's try first with row-based

Execute a lambda function for each row

In [17]:
%%time
arrow_ds.filter(lambda x: x['amount'] > 10000).take(3)

Read->Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:10<00:00,  2.19s/it]

CPU times: user 173 ms, sys: 62.4 ms, total: 236 ms
Wall time: 11 s


[ArrowRow({'id': 6667,
           'ssn': None,
           'name': None,
           'amount': 10000.5,
           'interest': 0.30000000000000004,
           'state': 'TX',
           'marital_status': 'undeclared',
           'property': 'condo',
           'dependents': 1,
           'defaulted': 0,
           'gender': 'U'}),
 ArrowRow({'id': 6668,
           'ssn': None,
           'name': None,
           'amount': 10002.0,
           'interest': 0.1,
           'state': 'CA',
           'marital_status': 'single',
           'property': 'house',
           'dependents': 2,
           'defaulted': 1,
           'gender': 'M'}),
 ArrowRow({'id': 6669,
           'ssn': None,
           'name': None,
           'amount': 10003.5,
           'interest': 0.5,
           'state': 'WA',
           'marital_status': 'divorced',
           'property': 'rental',
           'dependents': 4,
           'defaulted': 0,
           'gender': 'U'})]

Let's try a `.map_batches()`, which is vectorized 

In [18]:
%%time
arrow_ds.map_batches(lambda df: df[df["amount"] > 10000]).take(3)

Read->Map_Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10.49it/s]

CPU times: user 66.2 ms, sys: 20.7 ms, total: 86.8 ms
Wall time: 514 ms


[PandasRow({'id': 6667,
            'ssn': None,
            'name': None,
            'amount': 10000.5,
            'interest': 0.30000000000000004,
            'state': 'TX',
            'marital_status': 'undeclared',
            'property': 'condo',
            'dependents': 1,
            'defaulted': 0,
            'gender': 'U'}),
 PandasRow({'id': 6668,
            'ssn': None,
            'name': None,
            'amount': 10002.0,
            'interest': 0.1,
            'state': 'CA',
            'marital_status': 'single',
            'property': 'house',
            'dependents': 2,
            'defaulted': 1,
            'gender': 'M'}),
 PandasRow({'id': 6669,
            'ssn': None,
            'name': None,
            'amount': 10003.5,
            'interest': 0.5,
            'state': 'WA',
            'marital_status': 'divorced',
            'property': 'rental',
            'dependents': 4,
            'defaulted': 0,
            'gender': 'U'})]

Let's try a filter operation: both per row operation and per block as vectorized

In [19]:
%%time
arrow_ds.filter(lambda x: x['amount'] > 10000.00 and x['state'] == 'CA').take(2)

Read->Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.02it/s]

CPU times: user 92.2 ms, sys: 31.4 ms, total: 124 ms
Wall time: 4.91 s


[ArrowRow({'id': 6668,
           'ssn': None,
           'name': None,
           'amount': 10002.0,
           'interest': 0.1,
           'state': 'CA',
           'marital_status': 'single',
           'property': 'house',
           'dependents': 2,
           'defaulted': 1,
           'gender': 'M'}),
 ArrowRow({'id': 6670,
           'ssn': None,
           'name': None,
           'amount': 10005.0,
           'interest': 0.30000000000000004,
           'state': 'CA',
           'marital_status': 'divorced',
           'property': 'condo',
           'dependents': 1,
           'defaulted': 0,
           'gender': 'M'})]

In [20]:
%%time
arrow_ds.map_batches(lambda df: df[[df["amount"] > 10000] and df["state"] == "CA"]).take(3)

Read->Map_Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 18.52it/s]

CPU times: user 64.4 ms, sys: 17.3 ms, total: 81.7 ms
Wall time: 296 ms


[PandasRow({'id': 1,
            'ssn': None,
            'name': None,
            'amount': 1.5,
            'interest': 0.1,
            'state': 'CA',
            'marital_status': 'married',
            'property': 'rental',
            'dependents': 3,
            'defaulted': 1,
            'gender': 'F'}),
 PandasRow({'id': 2,
            'ssn': None,
            'name': None,
            'amount': 3.0,
            'interest': 0.4,
            'state': 'CA',
            'marital_status': 'divorced',
            'property': 'house',
            'dependents': 3,
            'defaulted': 0,
            'gender': 'M'}),
 PandasRow({'id': 20,
            'ssn': None,
            'name': None,
            'amount': 30.0,
            'interest': 0.2,
            'state': 'CA',
            'marital_status': 'undeclared',
            'property': 'house',
            'dependents': 3,
            'defaulted': 1,
            'gender': 'F'})]

Use `groupBy` state and compute the count

Under the hood is distributed parallel group and vectorized not using UDFs

In [21]:
results = arrow_ds.groupby("state").count()

Shuffle Reduce: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 670.23it/s]


In [22]:
results.show()

{'state': 'AZ', 'count()': 124939}
{'state': 'CA', 'count()': 124900}
{'state': 'OR', 'count()': 125353}
{'state': 'TX', 'count()': 124989}
{'state': 'UT', 'count()': 125524}
{'state': 'WA', 'count()': 124295}


Get the max of these columns

In [23]:
results=arrow_ds.max(["amount", "interest", "dependents"])
results

Shuffle Reduce: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 163.53it/s]


ArrowRow({'max(amount)': 1125000.0,
          'max(interest)': 0.5,
          'max(dependents)': 5})

### Accessing datasets using batches or iterating by rows

Datasets can be passed to Ray tasks or actors and read with `.iter_batches()` or `.iter_rows()`. This does not incur a copy, since the blocks of the Dataset are passed by reference as Ray objects. Splitting data as shards and passing to individual Ray Actors to process shards in a common Ray pattern used in distributed training with Ray actors.

Let's examine how.

*Questions for clearificaiton*:
 * _why is shard a list now instead of Dataset_?

In [24]:
@ray.remote
class BatchWorker:
    def __init__(self, rank):
        self.rank = rank
        self.processed= 0
    
    @ray.method(num_returns=2)
    def process_shard_list(self, shard: ray.data.Dataset) -> tuple:
        for batch in shard.iter_batches(batch_size=1024):
            # do something with the batch such as feature
            # processing, transformation, and 
            # save as a parquet files 
            self.processed = self.processed + len(batch)
        # return items processed, worker id
        return (self.processed, self.rank)     

#### Create batch workers as Ray actors
Each actor will get a shard, list of rows, to work on. We split
our dataset `arrow_ds` into five shards. Each `BatchWorker` gets a shard.
`.split`() splits shards across these batch of workers by using the `locality_hints`

In [25]:
batch_workers = [BatchWorker.remote(i) for i in range(1, 6)]

shards = arrow_ds.split(len(batch_workers), locality_hints=batch_workers)

print(f"Shard row: {shards[0]}")
print(f"Number of shards:{len(shards)}")
print(f"Number of shard workers:{len(batch_workers)}")

Read progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 80.13it/s]

Shard row: Dataset(num_blocks=1, num_rows=150000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string})
Number of shards:5
Number of shard workers:5


### Launch `BatchWorker` actors

Process each shard. Each `BatchWorker.process_shard_list()` returns a object RefID with a tuple as its value. What we get from this comprehension is a list objectRefs as tuples.

In [26]:
object_refs = [w.process_shard_list.remote(s) for w, s in zip(batch_workers, shards)]
object_refs, len(object_refs)

([[ObjectRef(e0513aa905843ccfe0c0b4b425af785995f22c5d0100000001000000),
   ObjectRef(e0513aa905843ccfe0c0b4b425af785995f22c5d0100000002000000)],
  [ObjectRef(5f70e045687d2f9ad5e2f517143196f1fa25b8e90100000001000000),
   ObjectRef(5f70e045687d2f9ad5e2f517143196f1fa25b8e90100000002000000)],
  [ObjectRef(a4dc031465f905f8359c6ce6c4a93403a78cf4e30100000001000000),
   ObjectRef(a4dc031465f905f8359c6ce6c4a93403a78cf4e30100000002000000)],
  [ObjectRef(9e7872a82e7456d9a9db65608fa756a9e738562f0100000001000000),
   ObjectRef(9e7872a82e7456d9a9db65608fa756a9e738562f0100000002000000)],
  [ObjectRef(cd25e647a728676bca8ad1bf20ca991c6cdb92590100000001000000),
   ObjectRef(cd25e647a728676bca8ad1bf20ca991c6cdb92590100000002000000)]],
 5)

Fetch the values from the returned list of ObjectRefs, which is a tuple of (batch_size, worker_rank).

In [27]:
values = [ray.get(ref) for ref in object_refs]
values

[[150000, 1], [150000, 2], [150000, 3], [150000, 4], [150000, 5]]

### Creating and using dataset pipelines

What are dataset pipelines and how are they different from Ray datasets? 

Datasets perform transformation or operations eagerly or synchronously, whereas [DataPipelines](https://docs.ray.io/en/latest/data/package-ref.html#datasetpipeline-api) can execute in an overlaped pipeline executions. For example, if you had operations that require reading from file, transforming data, and then doing some minor feature engineering, these operations can be executed in a normal pipeline fashion. This allows for the overlapped execution of data input (e.g., reading files), computation (e.g. feature preprocessing), and training (e.g., distributed ML training).

A `DatasetPipeline` can be constructed in two ways: either by pipelining the execution of an existing Dataset (via `Dataset.window`) or generating repeats of an existing Dataset (via `Dataset.repeat`). 

Let's have a go at it and see what we can do with our synthetic data from above.


### Using Dataset.window

Create some functions or operations to be executed in a overlapped manner in the pipeline. These functions
are simple to illustrate a point. But they can be complex for a particular use case.

 _Questions for clarification_:
 * _how can we send arguments to these pipeline functions?_

In [28]:
def divide_row_value(row: ray.data.impl.arrow_block.ArrowRow) -> int:
    return round(row / 2)

In [29]:
def double_row_value(row: ray.data.impl.arrow_block.ArrowRow) -> int:
    return row * 2

In [30]:
def modulo_row_value(row: ray.data.impl.arrow_block.ArrowRow) -> int:
    return row % random.randint(1, 42)

#### Create a window based pipeline
With a each window of 50 blocks. 

_Questions for clarification_:
 * _why the number of stages is 2?_

In [31]:
ds_pipe = ds.window(blocks_per_window=50)
ds_pipe

DatasetPipeline(num_windows=4, num_stages=2)

### Applying transforms to pipelines adds more pipeline stages.
_Questions for clarification_:
 * _how can we send arguments to thse pipeline functions?_

In [32]:
ds_pipe = ds_pipe.map(divide_row_value)
ds_pipe = ds_pipe.map(double_row_value)
ds_pipe = ds_pipe.map(modulo_row_value)
print(ds_pipe)

DatasetPipeline(num_windows=4, num_stages=5)


#### Iterate our pipeline

 * _Questions for clearification_:
     * _how is this executed_?
     * _why are we iterating over rows_?
     * _what is row comprised of? Blocks?_?
     * _is the value of the row an already computed value_?
     * _if the `num_stages=5`, why am I seeing only stage 0 and 1 in the output of stages?_

In [33]:
results=[]
for row in ds_pipe.iter_rows():
    results.append(row)
print(f"Total value: {sum(results)}")

Stage 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.67s/it]

Total value: 990185


Let's try a Datapipeline with our synthetic data

In [34]:
# count or return based on the condition
def count_ca(row: ray.data.impl.arrow_block.ArrowRow) -> int:
    return 1 if row['state'] == "CA" and row["defaulted"] else 0

In [35]:
arrow_ds_pipe = arrow_ds.window(blocks_per_window=50)
arrow_ds_pipe

DatasetPipeline(num_windows=1, num_stages=2)

In [36]:
arrow_ds_pipe = arrow_ds_pipe.map(count_ca)
arrow_ds_pipe

DatasetPipeline(num_windows=1, num_stages=3)

In [37]:
results=[]
for row in arrow_ds_pipe.iter_rows():
    results.append(row)
print(f"Total rows for CA state and defaulted loans rows: {sum(results)}")

Stage 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.61s/it]

Total rows for CA state and defaulted loans rows: 62209


### Ingesting into Model Trainers
Let's define a dummy trainer that takes our synthetic data and trains the model and returns loss for that trainer

In [38]:
def model(input):
    return random.uniform(0, 1)

@ray.remote
class Trainer:
    def __init__(self, rank, model):
        self.rank = rank
        self.model = model
        self.loss = 0.0
        
    def train(self, shard:ray.data.Dataset) -> float:
        for batch in shard.iter_batches(batch_size=1024):
            for epoch in range(1,21):
                output = self.model(batch)
                self.loss = output 
        if epoch % 5 == 0:
            print(f'epoch {epoch}, loss: {self.loss:.3f}')
        return self.loss

In [39]:
trainers = [Trainer.remote(i, model) for i in range(1, 6)]
trainers

[Actor(Trainer, b915a6e85f9918f2305bb73d01000000),
 Actor(Trainer, 0ba3cb456bf08dc534b9512401000000),
 Actor(Trainer, fce1e10840b442dd6c8ebd1a01000000),
 Actor(Trainer, fcbef636dbdc002beed3b30b01000000),
 Actor(Trainer, 17ed2149421241fcbbb7c7f501000000)]

In [40]:
shards = arrow_ds.split(n=len(trainers), locality_hints=trainers)
shards

Read progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 89.01it/s]


[Dataset(num_blocks=1, num_rows=150000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=1, num_rows=150000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=1, num_rows=150000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=1, num_rows=150000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=1, num_rows=150000, schema={id: int64, ssn: null, name: null, amount

In [41]:
object_refs = [t.train.remote(s) for t, s in zip(trainers, shards)]
ray.get(object_refs)

[0.10070883593326718,
 0.8285032544493941,
 0.7244037956232022,
 0.5640483846268071,
 0.5488837168820615]

In [42]:
ray.shutdown()

### Exercises
 1. Write some transformers, filters, and aggregators with our systhetic data
 2. Add additional pipleline stages with our synthetic data

### Homework

1. Work through the [NYC example tutorial](extra/ray_data_nyc.ipynb). This explores how you use `.map_batches()` for filtering and map operations using vectorized UDFs
2. Peruse the user guides for advanced examples in [data transformation](https://docs.ray.io/en/master/data/transforming-datasets.html#transforming-datasets) and [ML preprocessing](https://docs.ray.io/en/master/data/dataset-ml-preprocessing.html#datasets-ml-preprocessing)
3. Read how to do large scale [ML ingest](https://docs.ray.io/en/master/data/examples/big_data_ingestion.html)